In [28]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mc
import statsmodels.formula.api as sm
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
# Import the RandomForestClassifier from sklearn.ensemble
from sklearn.feature_extraction.text import CountVectorizer
random_forest = RandomForestClassifier()
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from imblearn.under_sampling import ClusterCentroids
from sklearn.metrics import accuracy_score

In [2]:
us_df = pd.read_csv('USvideos.csv')

In [3]:
# Remove unecessary columns
del_col = ['video_id','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description']
us_df = us_df.drop(del_col, axis=1)

In [4]:
us_df['tags'].head(50)

0                                       SHANtell martin
1     last week tonight trump presidency|"last week ...
2     racist superman|"rudy"|"mancuso"|"king"|"bach"...
3     rhett and link|"gmm"|"good mythical morning"|"...
4     ryan|"higa"|"higatv"|"nigahiga"|"i dare you"|"...
5     ijustine|"week with iPhone X"|"iphone x"|"appl...
6     SNL|"Saturday Night Live"|"SNL Season 43"|"Epi...
7     5 Ice Cream Gadgets|"Ice Cream"|"Cream Sandwic...
8     Trailer|"Hugh Jackman"|"Michelle Williams"|"Za...
9     vox.com|"vox"|"explain"|"shift change"|"future...
10    NFL|"Football"|"offense"|"defense"|"afc"|"nfc"...
11    The Walking Dead|"shiva"|"tiger"|"king ezekiel...
12    marshmello|"blocks"|"marshmello blocks"|"block...
13    nowthis|"nowthis world"|"world news"|"nowthis ...
14    shopping for new fish|"new fish"|"aquarium fis...
15    Robots|"Boston Dynamics"|"SpotMini"|"Legged Lo...
16    pacific rim|"pacific rim 2"|"pacific rim seque...
17    TED|"TED-Ed"|"TED Education"|"TED Ed"|"Hil

In [5]:
us_df['tags'].unique().shape[0]

6055

In [6]:
all_tags = []
for tgs in us_df['tags']:
    if tgs == '[none]': continue
    all_tags.extend(tgs.split('|'))
tags_freq = pd.Series(all_tags).value_counts().to_dict()

In [7]:
list(tags_freq.keys())

['"funny"',
 '"comedy"',
 '"how to"',
 '"Pop"',
 '"2018"',
 '"music"',
 '"humor"',
 '"food"',
 '"science"',
 '"review"',
 '"news"',
 '"vlog"',
 '"makeup"',
 '"celebrity"',
 '"tutorial"',
 '"live"',
 '"comedian"',
 '"interview"',
 '"video"',
 '"cooking"',
 '"television"',
 '"celebrities"',
 '"fun"',
 '"movie"',
 '"cute"',
 '"trailer"',
 '"Comedy"',
 '"family friendly"',
 '"official"',
 '"hollywood"',
 '"funny video"',
 '"NBC"',
 '"entertainment"',
 '"Records"',
 '"challenge"',
 '"2017"',
 '"sports"',
 '"education"',
 '"jokes"',
 '"animation"',
 '"late night"',
 '"recipe"',
 '"beauty"',
 '"animals"',
 '"basketball"',
 '"makeup tutorial"',
 '"sketch"',
 '"game"',
 '"Trailer"',
 '"fashion"',
 '"best"',
 '"music video"',
 '"talk show"',
 '"diy"',
 '"documentary"',
 '"Funny"',
 '"reaction"',
 '"technology"',
 '"show"',
 '"movies"',
 '"family"',
 '"hilarious"',
 '"new"',
 '"funny videos"',
 '"games"',
 '"clip"',
 '"highlights"',
 '"drama"',
 '"film"',
 '"amazing"',
 '"chef"',
 '"christmas"',


In [8]:
top_keys = []
for i, key in enumerate(list(tags_freq.keys())):
    key = key.replace('"', '')
    if i <100:
        top_keys.append(key)
        
top_keys

['funny',
 'comedy',
 'how to',
 'Pop',
 '2018',
 'music',
 'humor',
 'food',
 'science',
 'review',
 'news',
 'vlog',
 'makeup',
 'celebrity',
 'tutorial',
 'live',
 'comedian',
 'interview',
 'video',
 'cooking',
 'television',
 'celebrities',
 'fun',
 'movie',
 'cute',
 'trailer',
 'Comedy',
 'family friendly',
 'official',
 'hollywood',
 'funny video',
 'NBC',
 'entertainment',
 'Records',
 'challenge',
 '2017',
 'sports',
 'education',
 'jokes',
 'animation',
 'late night',
 'recipe',
 'beauty',
 'animals',
 'basketball',
 'makeup tutorial',
 'sketch',
 'game',
 'Trailer',
 'fashion',
 'best',
 'music video',
 'talk show',
 'diy',
 'documentary',
 'Funny',
 'reaction',
 'technology',
 'show',
 'movies',
 'family',
 'hilarious',
 'new',
 'funny videos',
 'games',
 'clip',
 'highlights',
 'drama',
 'film',
 'amazing',
 'chef',
 'christmas',
 'kitchen',
 'kids',
 'joke',
 'love',
 'vlogger',
 'CBS',
 'comedic',
 'celeb',
 'host',
 'health',
 'animated',
 'famous',
 'parody',
 'cosmet

In [9]:
model_df = us_df[['tags', 'views']]
model_df

,tags,views
0,SHANtell martin,748374
1,"last week tonight trump presidency|""last week ...",2418783
2,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434
3,"rhett and link|""gmm""|""good mythical morning""|""...",343168
4,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731
...,...,...
40944,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609
40945,[none],1064798
40946,I gave safiya nygaard a perfect hair makeover ...,1066451
40947,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813


In [10]:
for key in top_keys:
    col_name = 'has_' + key + '_tag'
    model_df[col_name] = 0
    for index, row, in model_df.iterrows():
        tags = row[0]
        if key in tags:
            model_df[col_name][index] = 1
            
model_df

C:\Users\Andy\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Andy\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Andy\anaconda3\envs\py3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

,tags,views,has_funny_tag,has_comedy_tag,has_how to_tag,has_Pop_tag,has_2018_tag,has_music_tag,has_humor_tag,has_food_tag,...,has_lol_tag,has_laugh_tag,has_pop_tag,has_cook_tag,has_unboxing_tag,has_song_tag,has_pets_tag,has_dance_tag,has_talk_tag,has_Music_tag
0,SHANtell martin,748374,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"last week tonight trump presidency|""last week ...",2418783,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"rhett and link|""gmm""|""good mythical morning""|""...",343168,1,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40945,[none],1064798,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40946,I gave safiya nygaard a perfect hair makeover ...,1066451,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40947,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y = model_df[['views']]
X = model_df.drop(['views', 'tags'], axis = 1)

In [12]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",random_state=1)

In [ ]:
# Train the model
log_classifier.fit(X_train_scaled,y_train.values.ravel())

In [ ]:
# # Evaluate the model
# y_pred = log_classifier.predict(X_test_scaled)
# print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")